In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import math
import numpy as np
import pandas as pd
device = 'cuda'

In [ ]:

sensor_groups = {
    'fan': [7, 12, 0, 4, 17, 18],
    'LPC': [1],
    'HPC': [2, 6, 10], 
    'core': [8, 13], 
    'turbine': [3, 19, 20], 
    'other': [11, 14, 16, 5, 9, 15] 
}

class GroupEncoder(nn.Module):
    """Encoder module for each sensor group"""
    def __init__(self, input_dim, hidden_dim, dropout_rate=0.2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            #nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            #nn.Dropout(dropout_rate)
        )
        
    def forward(self, x):
        return self.net(x)

class GroupDecoder(nn.Module):
    """Decoder module for each sensor group"""
    def __init__(self, output_dim, latent_dim, hidden_dim, dropout_rate=0.2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, output_dim),
            #nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.net(x)

class GroupVAE(nn.Module):
    def __init__(self, sensor_groups, hidden_dim=10, latent_dim=2, dropout_rate=0.2, device='cpu'):
        super().__init__()
        self.sensor_groups = sensor_groups
        self.num_groups = len(sensor_groups)
        self.hidden_dim = hidden_dim
        self.latent_dim = latent_dim
        self.device = device

        # Encoders for each group
        self.encoders = nn.ModuleDict({
            group: GroupEncoder(len(indices), hidden_dim, dropout_rate)
            for group, indices in sensor_groups.items()
        })

        # Latent space parameters
        self.fc_mu = nn.Linear(hidden_dim * self.num_groups, latent_dim)
        self.fc_log_var = nn.Linear(hidden_dim * self.num_groups, latent_dim)

        # Decoders for each group
        self.decoders = nn.ModuleDict({
            group: GroupDecoder(len(indices), latent_dim, hidden_dim, dropout_rate)
            for group, indices in sensor_groups.items()
        })

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        # Encode each group
        encoded = []
        for group, indices in self.sensor_groups.items():
            encoded.append(self.encoders[group](x[:, indices]))
        
        # Concatenate and get latent parameters
        h_concat = torch.cat(encoded, dim=1)
        mu = self.fc_mu(h_concat)
        log_var = self.fc_log_var(h_concat)
        z = self.reparameterize(mu, log_var)
        
        # Decode each group
        decoded = []
        for group in self.sensor_groups.keys():
            decoded.append(self.decoders[group](z))
            
        return torch.cat(decoded, dim=1), mu, log_var

    def mc_dropout_sample(self, x, num_samples=50):
        """Monte Carlo sampling with dropout enabled"""
        samples = []
        self.train()
        with torch.no_grad():
            for _ in range(num_samples):
                recon, _, _ = self(x)
                samples.append(recon)
        return torch.stack(samples)


    def quantify_uncertainty(self, x, num_samples=50):
        """Returns group-wise and global uncertainty estimates"""
        x = x.to(device)
        mc_samples = self.mc_dropout_sample(x, num_samples)
    
        # Global uncertainties if you want to test with them vs by individual groups
        reconstruction_errors_global = torch.linalg.norm(x - mc_samples, dim=2)**2  # (samples, N)
        epistemic_global = torch.var(reconstruction_errors_global, dim=0)
    
        mean_recon = mc_samples.mean(dim=0)
        aleatoric_global = torch.mean((mc_samples - mean_recon.unsqueeze(0))**2, dim=(0, 2))
    
        # Group-wise uncertainties
        group_aleatoric = {}
        group_epistemic = {}
    
        for group, indices in self.sensor_groups.items():
            # Slice features for this group
            group_samples = mc_samples[:, :, indices]
    
            # Compute reconstruction errors for the group
            reconstruction_errors_group = torch.linalg.norm(x[:, indices] - group_samples, dim=2)**2  # (samples, N)
    
            # Calculate group-specific uncertainties
            group_epistemic[group] = torch.var(reconstruction_errors_group, dim=0)
            group_mean = group_samples.mean(dim=0)
            group_aleatoric[group] = torch.mean(
                (group_samples - group_mean.unsqueeze(0))**2,
                dim=(0, 2)
            )
    
        return {
            'global': (aleatoric_global.cpu().numpy(), epistemic_global.cpu().numpy()),
            'groups': (
                {k: v.cpu().numpy() for k, v in group_aleatoric.items()},
                {k: v.cpu().numpy() for k, v in group_epistemic.items()}
            )
        }

    

# Modified activation retrieval functions
def get_group_encoder_activations(encoder, data):
    """Get activations from a single group encoder"""
    activations = []
    x = data
    for layer in encoder.net:
        if isinstance(layer, nn.Linear):
            x = layer(x)
            activations.append(x.detach())
        else:
            x = layer(x)
    return activations

def get_shared_latent_activation(model, data):
    """Get deterministic latent representation (mu)"""
    with torch.no_grad():
        encoded = []
        for group, indices in model.sensor_groups.items():
            group_input = data[:, indices]
            group_enc = model.encoders[group](group_input)
            encoded.append(group_enc)
        h_concat = torch.cat(encoded, dim=1)
        mu = model.fc_mu(h_concat)
    return mu.detach()


# Corrected RaPP calculation functions
def calculate_encoder_rapp(model, train_loader, full_data, scaler):
    """Calculate group-wise encoder RaPP metrics"""
    encoder_sap = {}
    encoder_nap = {}
    
    # SAP Calculation
    with torch.no_grad():
        X_all = torch.FloatTensor(scaler.transform(full_data)).to(device)
        recon_all, _, _ = model(X_all)
        
        for group, indices in model.sensor_groups.items():
            group_input = X_all[:, indices]
            group_recon = recon_all[:, indices]
            
            orig_acts = get_group_encoder_activations(model.encoders[group], group_input)
            recon_acts = get_group_encoder_activations(model.encoders[group], group_recon)
            
            sap_layers = [torch.sqrt(torch.sum((orig - rec)**2, dim=1)) 
                         for orig, rec in zip(orig_acts, recon_acts)]
            encoder_sap[group] = torch.sum(torch.stack(sap_layers), dim=0).cpu().numpy()

    # NAP Calculation
    model.eval()
    D_train = {group: [] for group in model.sensor_groups}
    with torch.no_grad():
        for batch_X, _ in train_loader:
            batch_X = batch_X.to(device)
            recon_batch, _, _ = model(batch_X)
            
            for group, indices in model.sensor_groups.items():
                group_input = batch_X[:, indices]
                group_recon = recon_batch[:, indices]
                
                orig_acts = get_group_encoder_activations(model.encoders[group], group_input)
                recon_acts = get_group_encoder_activations(model.encoders[group], group_recon)
                
                diffs = [orig - rec for orig, rec in zip(orig_acts, recon_acts)]
                flattened_diffs = torch.cat([d.flatten(start_dim=1) for d in diffs], dim=1)
                D_train[group].append(flattened_diffs)

    # Process each group's NAP
    for group in model.sensor_groups:
        D_train[group] = torch.cat(D_train[group], dim=0)
        D_mean = D_train[group].mean(dim=0)
        D_centered = D_train[group] - D_mean
        
        U, S, V = torch.linalg.svd(D_centered, full_matrices=False)
        
        with torch.no_grad():
            X_full = torch.FloatTensor(scaler.transform(full_data)).to(device)
            recon_full, _, _ = model(X_full)
            
            group_input = X_full[:, model.sensor_groups[group]]
            group_recon = recon_full[:, model.sensor_groups[group]]
            
            orig_acts = get_group_encoder_activations(model.encoders[group], group_input)
            recon_acts = get_group_encoder_activations(model.encoders[group], group_recon)
            
            diffs = [orig - rec for orig, rec in zip(orig_acts, recon_acts)]
            D = torch.cat([d.flatten(start_dim=1) for d in diffs], dim=1)
            D = D - D_mean
            
            Sigma_inv = torch.diag(1.0/(S + 1e-6))
            proj = D @ V.T @ Sigma_inv
            encoder_nap[group] = torch.norm(proj, dim=1).cpu().numpy()

    return encoder_sap, encoder_nap
           

def calculate_latent_rapp(model, train_loader, full_data, scaler):
    """Calculate shared latent space RaPP metrics"""
    # SAP Calculation
    with torch.no_grad():
        X_all = torch.FloatTensor(scaler.transform(full_data)).to(device)
        mu_original = get_shared_latent_activation(model, X_all)
        recon_all, _, _ = model(X_all)
        mu_recon = get_shared_latent_activation(model, recon_all)
        
    sap = torch.sqrt(torch.sum((mu_original - mu_recon)**2, dim=1)).cpu().numpy()

    # NAP Calculation
    model.eval()
    D_train = []
    with torch.no_grad():
        for batch_X, _ in train_loader:
            batch_X = batch_X.to(device)
            mu_orig = get_shared_latent_activation(model, batch_X)
            recon_batch, _, _ = model(batch_X)
            mu_recon = get_shared_latent_activation(model, recon_batch)
            D_train.append(mu_orig - mu_recon)
    
    D_train = torch.cat(D_train, dim=0)
    D_mean = D_train.mean(dim=0)
    D_centered = D_train - D_mean
    
    U, S, V = torch.linalg.svd(D_centered, full_matrices=False)
    
    with torch.no_grad():
        X_full = torch.FloatTensor(scaler.transform(full_data)).to(device)
        mu_orig = get_shared_latent_activation(model, X_full)
        recon_full, _, _ = model(X_full)
        mu_recon = get_shared_latent_activation(model, recon_full)
        
        D = (mu_orig - mu_recon) - D_mean
        
        Sigma_inv = torch.diag(1.0/(S + 1e-6))
        proj = D @ V.T @ Sigma_inv
        nap = torch.norm(proj, dim=1).cpu().numpy()

    return sap, nap


# Compute RUL for each unit
# Fixed RUL calculation function
def calculate_rul(data):
    # Get unique units using dataset_id (column 0) and unit_num (column 1)
    unique_units = np.unique(data[:, [0, 1]], axis=0)
    
    for dataset_id, unit_num in unique_units:
        # Create mask for this specific unit
        mask = (data[:, 0] == dataset_id) & (data[:, 1] == unit_num)
        unit = data[mask]
        
        # Skip if no data found (shouldn't happen with proper input)
        if unit.size == 0:
            continue
            
        # Calculate RUL components
        max_cycle = np.max(unit[:, 2])  # Max cycle for this unit
        rul_extra = dataset_test_RUL[int(unit_num)-1] if dataset_id == 1 else 0
        rul_values = (max_cycle - unit[:, 2]) + rul_extra
        
        # Update RUL in original data
        data[mask, 2] = rul_values
        
    return data

# Process datasets to retain unit numbers
def process_dataset(data, unit_id=None):
    if unit_id is not None:
        # Filter the data to include only the entries with the specified unit_id
        data = data[data[:, 0] == unit_id]

    return np.hstack((
        data[:, 0].reshape(-1, 1),   # Unit number
        data[:, 1].reshape(-1, 1),   # Cycles
        data[:, 5:]                  # Sensors
    ))

# Create sliding windows
def create_windows(data, window_size=30, step=1, threshold=125):
    windows = []
    unique_units = np.unique(data[:, :2], axis=0)
    for dataset_id, unit_num in unique_units:
        mask = (data[:, 0] == dataset_id) & (data[:, 1] == unit_num)
        unit_data = data[mask]
        n_samples = len(unit_data)
        for i in range(0, n_samples - window_size + 1, step):
            window = unit_data[i:i+window_size, 3:]
            target_rul = unit_data[i+window_size-1, 2]
            if target_rul > threshold:
                windows.append(window.flatten())
    return np.array(windows)
    

# 1. Corrected data processing without windowing
def create_full_samples(data, is_test=False, clip_threshold=None):
    samples = []
    ruls = []
    unique_units = np.unique(data[:, 0])  # Get unique unit numbers

    for unit_num in unique_units:
        mask = data[:, 0] == unit_num
        unit_data = data[mask]

        max_cycle = np.max(unit_data[:, 1])  # Max cycle for this unit
        rul_extra = dataset_test_RUL[int(unit_num)-1] if is_test else 0

        for i in range(len(unit_data)):
            sample = unit_data[i, 2:]  # Sensor measurements
            target_rul = (max_cycle - unit_data[i, 1]) + rul_extra

            if clip_threshold is not None:
                target_rul = min(target_rul, clip_threshold)

            samples.append(sample)
            ruls.append(target_rul)

    return np.array(samples), np.array(ruls)

# 2. Create features from original model
def create_feature_df(model, X, ruls, train_loader, scaler):
    with torch.no_grad():
        X_tensor = torch.FloatTensor(scaler.transform(X))
        uncertainties = model.quantify_uncertainty(X_tensor)
    
    # Unpack uncertainties
    aleatoric_global, epistemic_global = uncertainties['global']
    aleatoric_groups, epistemic_groups = uncertainties['groups']

    encoder_sap, encoder_nap = calculate_encoder_rapp(model, train_loader, X, scaler)
    latent_sap, latent_nap = calculate_latent_rapp(model, train_loader, X, scaler)

    metrics_df = pd.DataFrame({
        'Aleatoric': aleatoric_global,
        'Epistemic': epistemic_global,
        'Latent_SAP': latent_sap,
        'Latent_NAP': latent_nap,
        'RUL': ruls
    })

    # Add encoder-specific metrics
    for group in sensor_groups.keys():
        metrics_df[f'Encoder_{group}_SAP'] = encoder_sap[group]
        metrics_df[f'Encoder_{group}_NAP'] = encoder_nap[group]
        metrics_df[f'Aleatoric_{group}'] = aleatoric_groups[group]
        metrics_df[f'Epistemic_{group}'] = epistemic_groups[group]

    return metrics_df

def train_evaluate_rul_model(X_train_copy, y_train_copy, X_test_copy, y_test_copy, patterns_to_keep, RUL_PARAMS, RF_PARAMS):
    """
    Train and evaluate a RandomForestRegressor model for RUL prediction.

    Parameters:
    - X_train_copy: DataFrame, training features
    - y_train_copy: Series, training target
    - X_test_copy: DataFrame, test features
    - y_test_copy: Series, test target
    - patterns_to_keep: List of strings, patterns to keep in the features
    - RUL_PARAMS: Dictionary, parameters for RUL processing
    - RF_PARAMS: Dictionary, parameters for RandomForestRegressor

    Returns:
    - trained_model: Trained RandomForestRegressor model
    - mse: Mean Squared Error on the full test set
    - mae: Mean Absolute Error on the full test set
    - mse_last: Mean Squared Error on the test_last set
    - mae_last: Mean Absolute Error on the test_last set
    """

    # Filter columns based on patterns
    columns_to_keep_train = [col for col in X_train_copy.columns if any(pattern in col for pattern in patterns_to_keep)]
    X_train = X_train_copy[columns_to_keep_train].copy()
    columns_to_keep_test = [col for col in X_test_copy.columns if any(pattern in col for pattern in patterns_to_keep)]
    X_test = X_test_copy[columns_to_keep_test].copy()

    # Normalize the data
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    def train_evaluate_model(X_train, y_train, X_test, y_test):
        # Handle RUL modifications
        if RUL_PARAMS['clip_threshold']:
            y_train = np.clip(y_train, None, RUL_PARAMS['clip_threshold'])
            y_test = np.clip(y_test, None, RUL_PARAMS['clip_threshold'])
            # print(f"Clipped RUL values at {RUL_PARAMS['clip_threshold']}")

        if RUL_PARAMS['filter_threshold']:
            train_mask = y_train < RUL_PARAMS['filter_threshold']
            test_mask = y_test < RUL_PARAMS['filter_threshold']
            X_train, y_train = X_train[train_mask], y_train[train_mask]
            X_test, y_test = X_test[test_mask], y_test[test_mask]
            # print(f"Filtered RUL >= {RUL_PARAMS['filter_threshold']}")

        # Initialize model
        rf = RandomForestRegressor(
            n_estimators=RF_PARAMS['n_estimators'],
            max_depth=RF_PARAMS['max_depth'],
            random_state=RF_PARAMS['random_state']
        )

        # Hyperparameter tuning
        if RUL_PARAMS['tune_hyperparams']:
            # print("Tuning hyperparameters...")
            grid_search = GridSearchCV(
                estimator=rf,
                param_grid=RF_PARAMS['param_grid'],
                cv=5,
                scoring='neg_mean_absolute_error',
                n_jobs=-1
            )
            grid_search.fit(X_train, y_train)
            best_rf = grid_search.best_estimator_
            print(f"Best parameters: {grid_search.best_params_}")
        else:
            # print("Using default parameters")
            best_rf = rf
            best_rf.fit(X_train, y_train)

        # Evaluate on the full test set
        y_pred = best_rf.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        # Create test_last dataset
        test_last_indices = np.where(np.diff(y_test, prepend=y_test[0]+1) > 0)[0]
        X_test_last = X_test[test_last_indices]
        y_test_last = y_test[test_last_indices]

        # Evaluate on the test_last set
        y_pred_last = best_rf.predict(X_test_last)
        mse_last = mean_squared_error(y_test_last, y_pred_last)
        mae_last = mean_absolute_error(y_test_last, y_pred_last)
        r2_last = r2_score(y_test_last, y_pred_last)

        return best_rf, mse, mae, mse_last, mae_last

    # Run training and evaluation
    trained_model, mse, mae, mse_last, mae_last = train_evaluate_model(
        X_train_scaled,
        y_train_copy.values,
        X_test_scaled,
        y_test_copy.values
    )

    # Feature importance visualization
    feature_importance = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': trained_model.feature_importances_
    }).sort_values('Importance', ascending=False)

    # print("\nFeature Importances:")
    # print(feature_importance)

    print(f'| {RUL_PARAMS['clip_threshold']} | {RUL_PARAMS['filter_threshold']} | {math.sqrt(mse):.2f} | {mae:.2f} | {math.sqrt(mse_last):.2f} | {mae_last:.2f} | {feature_importance.iloc[0]['Feature']} {feature_importance.iloc[0]['Importance']:.2f} | {feature_importance.iloc[1]['Feature']} {feature_importance.iloc[1]['Importance']:.2f} |', end='')
    if len(feature_importance) > 2:
        print(f'{feature_importance.iloc[2]['Feature']} {feature_importance.iloc[2]['Importance']:.2f} |')
    else:
        print(f'- |')

    rmse = math.sqrt(mse)
    rmse_last = math.sqrt(mse_last)

    return trained_model, rmse, rmse_last


In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch.optim as optim

def make_experiment_run(make_model=False):
    
    # Load datasets
    dataset_train = np.loadtxt(f'data/train_{dataset_name}.txt')
    dataset_test = np.loadtxt(f'data/test_{dataset_name}.txt')
    dataset_test_RUL = np.loadtxt(f'data/RUL_{dataset_name}.txt')
    
    
    train_data = process_dataset(dataset_train)
    test_data = process_dataset(dataset_test)
    
    full_data = np.vstack((
        np.hstack((np.zeros((train_data.shape[0], 1)), train_data)),
    ))

    full_data_test = np.vstack((
        np.hstack((np.zeros((test_data.shape[0], 1)), test_data)),
    ))
    
    
    # Apply corrected RUL calculation
    full_data = calculate_rul(full_data)
    full_data_test = calculate_rul(full_data_test)

    
    window_size = 1
    R_early_train = 80
    X = create_windows(full_data, window_size=window_size, threshold=R_early_train)
    X_test = create_windows(full_data_test, window_size=window_size, threshold=R_early_train)
    
    # Train-validation split
    X_train = X
    X_val = X_test
    
    # Scaling
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    len(train_data), len(X_train_scaled)
    
    ### INIT MODEL
    input_dim = X_train_scaled.shape[1]
    model = GroupVAE(
        sensor_groups=sensor_groups,
        hidden_dim=2,
        latent_dim=3,
        dropout_rate=0.2,
        device='cpu'
    )
    optimizer = torch.optim.Adam(model.parameters())
    criterion = torch.nn.MSELoss()
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    
    ### TRAIN MODEL
    # Convert to tensors
    train_data = TensorDataset(torch.FloatTensor(X_train_scaled), 
                              torch.FloatTensor(X_train_scaled))
    val_data = TensorDataset(torch.FloatTensor(X_val_scaled),
                            torch.FloatTensor(X_val_scaled))
    
    # Data loaders
    batch_size = 128
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)#, num_workers=12)
    val_loader = DataLoader(val_data, batch_size=batch_size)#, num_workers=12)

    model.to(device)
    # Training
    for epoch in range(200):
        # KL annealing 
        current_KL_coef = min(1.0, epoch / 50)  # Now actually using annealing
        
        model.train()
        train_loss = 0
        train_recon = 0
        train_kl = 0
        
        for batch_X, _ in train_loader:
            batch_X = batch_X.to(device)
            optimizer.zero_grad()
            
            # Forward pass - same interface as original VAE
            recon_batch, mu, log_var = model(batch_X)
            
            # Reconstruction loss (same as before)
            recon_loss = criterion(recon_batch, batch_X)
            
            # KL Divergence calculation (adjusted for correct scaling)
            kl_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp()) / batch_X.size(0)
            
            # Combined loss with annealing
            loss = recon_loss + kl_loss * current_KL_coef
            
            # Backpropagation
            loss.backward()
            optimizer.step()
            
            # Accumulate metrics
            train_loss += loss.item()
            train_recon += recon_loss.item()
            train_kl += kl_loss.item()
    
        # Validation phase
        model.eval()
        val_loss = 0
        val_recon = 0
        val_kl = 0
        
        with torch.no_grad():
            for batch_X, _ in val_loader:
                batch_X = batch_X.to(device)
                recon_val, mu_val, log_var_val = model(batch_X)
                
                # Validation reconstruction loss
                recon_loss_val = criterion(recon_val, batch_X)
                
                # KL loss (same calculation as training)
                kl_loss_val = -0.5 * torch.sum(1 + log_var_val - mu_val.pow(2) - log_var_val.exp()) / batch_X.size(0)
                
                # Validation loss (using same KL coefficient)
                loss_val = recon_loss_val + kl_loss_val * current_KL_coef
                
                val_loss += loss_val.item()
                val_recon += recon_loss_val.item()
                val_kl += kl_loss_val.item()
    
        # Step the scheduler
        scheduler.step()
    
        # Calculate epoch metrics
        train_recon_avg = train_recon / len(train_loader)
        train_kl_avg = train_kl / len(train_loader)
        train_loss_avg = train_loss / len(train_loader)
    
        val_loss_avg = val_loss / len(val_loader)
        val_recon_avg = val_recon / len(val_loader)
        val_kl_avg = val_kl / len(val_loader)

    print(f'Epoch {epoch+1:03d}: '
          f'Train Loss: {train_loss_avg:.4f} (Recon: {train_recon_avg:.4f}, KL: {train_kl_avg:.4f}) | '
          f'Val Loss: {val_loss_avg:.4f} (Recon: {val_recon_avg:.4f}, KL: {val_kl_avg:.4f})')

    model.eval()
    
    ### CREATE OUR METRICS FOR RANDOM FOREST
    # Load datasets
    dataset_train = np.loadtxt(f'data/train_{dataset_name}.txt')
    dataset_test = np.loadtxt(f'data/test_{dataset_name}.txt')
    dataset_test_RUL = np.loadtxt(f'data/RUL_{dataset_name}.txt')
    
    train_data = process_dataset(dataset_train)
    test_data = process_dataset(dataset_test)
    
    # 3. Full pipeline
    # Process raw data without thresholds
    train_samples, train_ruls = create_full_samples(train_data, is_test=False)
    test_samples, test_ruls = create_full_samples(test_data, is_test=True)
    
    # Scaling (use same scaler as before)
    scaler = MinMaxScaler().fit(train_samples)
    
    # Create feature DataFrames
    train_feats_df = create_feature_df(model, train_samples, train_ruls, train_loader, scaler)
    test_feats_df = create_feature_df(model, test_samples, test_ruls, train_loader, scaler)
    
    # 4. Train/test split
    X_train = train_feats_df.drop('RUL', axis=1)
    y_train = train_feats_df['RUL']
    X_test = test_feats_df.drop('RUL', axis=1)
    y_test = test_feats_df['RUL']
    
    init = True
    
    if init:
        X_train_copy = X_train.copy()
        y_train_copy = y_train.copy()
        X_test_copy = X_test.copy()
        y_test_copy = y_test.copy()
        init = False
    
    # Now train multiple RFs
    RUL_PARAMS = {
        'clip_threshold': 125,    # Set to None to disable clipping
        'filter_threshold': False, # Set to value (e.g. 125) to filter RUL >= threshold
        'tune_hyperparams': True  # Set to False to use default parameters
    }
    RF_PARAMS = {
    'n_estimators': 100,
    'max_depth': 15,
    'random_state': 42,
    'param_grid': {
        'n_estimators': [100],
        'max_depth': [10],
        'min_samples_split': [2]
        }
    }
    
    # Define the parameters to iterate over
    filter_thresholds = [False]
    patterns_list = [
        ['Encoder', 'Latent', 'Aleatoric_', 'Epistemic_'],
        ['Latent'],
        ['Encoder'],
        ['Aleatoric', 'Epistemic']
    ]
    
    # Initialize a DataFrame to store the results
    results = []
    
    # Loop over the parameter combinations
    for filter_threshold in filter_thresholds:
        for patterns_to_keep in patterns_list:
            # Update RUL_PARAMS
            RUL_PARAMS_copy = RUL_PARAMS.copy()
            RUL_PARAMS_copy['filter_threshold'] = filter_threshold
    
            # Train and evaluate the model
            trained_model, rmse, rmse_last = train_evaluate_rul_model(
                X_train_copy, y_train_copy, X_test_copy, y_test_copy, patterns_to_keep, RUL_PARAMS_copy, RF_PARAMS
            )
    
            # Store the results
            results.append({
                'filter_threshold': filter_threshold,
                'patterns_to_keep': patterns_to_keep,
                'rmse': rmse,
                'rmse_last': rmse_last,
            })
    
    # Convert the results to a DataFrame
    results_df = pd.DataFrame(results)
    
    # Display the DataFrame
    print(results_df)
    return results_df

In [ ]:
import numpy as np
import os

dataset_names = ['FD001']#, 'FD002', 'FD003', 'FD004']
device = 'cpu'
model_name = 'I-GLIDE_VAE'
num_runs = 10

for dataset_name in dataset_names:
    print(f'Processing dataset {dataset_name}')

    dataset_test_RUL = np.loadtxt(f'data/RUL_{dataset_name}.txt')
    csv_path = f'benchmarks/{model_name}_{dataset_name}.csv'
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    
    
    all_exp_results = []
    
    for experiment_run in range(num_runs):
        print(f'## Doing run {experiment_run+1}')
        results = make_experiment_run()
        all_exp_results.append(results)
    
    # Concatenate all results into a single DataFrame
    all_results_df = pd.concat(all_exp_results, ignore_index=True)
    
    # Save or append results to the CSV file
    if os.path.exists(csv_path):
        # Append to the existing file
        all_results_df.to_csv(csv_path, mode='a', header=False, index=False)
    else:
        # Create a new file
        all_results_df.to_csv(csv_path, index=False)
    
    print(f'Results saved to {csv_path}')

# Make some plots of the HIs

In [ ]:
# Train a HI model
import numpy as np
import os

dataset_name = 'FD001'
device = 'cpu'
num_runs = 1

print(f'Processing dataset {dataset_name}')

dataset_test_RUL = np.loadtxt(f'data/RUL_{dataset_name}.txt')
model = make_experiment_run(make_model=True)
all_exp_results.append(results)
model

In [ ]:
dataset_train = np.loadtxt(f'data/train_{dataset_name}.txt')
dataset_test = np.loadtxt(f'data/test_{dataset_name}.txt')
dataset_test_RUL = np.loadtxt(f'data/RUL_{dataset_name}.txt')
batch_size=1

# 1. Initial dataloaders to compute NAP
train_data = process_dataset(dataset_train)
test_data = process_dataset(dataset_test)

full_data = np.vstack((
    np.hstack((np.zeros((train_data.shape[0], 1)), train_data)),
))

full_data_test = np.vstack((
    np.hstack((np.zeros((test_data.shape[0], 1)), test_data)),
))


# Apply corrected RUL calculation
full_data = calculate_rul(full_data)
full_data_test = calculate_rul(full_data_test)

window_size = 1
R_early_train = 125
X = create_windows(full_data, window_size=window_size, threshold=R_early_train)
X_test = create_windows(full_data_test, window_size=window_size, threshold=R_early_train)

X_train = X
X_val = X_test

# Scaling
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
len(train_data), len(X_train_scaled)

### INIT MODEL
input_dim = X_train_scaled.shape[1]
model = GroupVAE(
    sensor_groups=sensor_groups,
    hidden_dim=2,
    latent_dim=3,
    dropout_rate=0.2,
    device='cpu'
)
optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.MSELoss()


### TRAIN MODEL
# Convert to tensors
train_data = TensorDataset(torch.FloatTensor(X_train_scaled), 
                      torch.FloatTensor(X_train_scaled))
val_data = TensorDataset(torch.FloatTensor(X_val_scaled),
                    torch.FloatTensor(X_val_scaled))


train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)#, num_workers=12)

train_data = process_dataset(dataset_train)
train_samples, train_ruls = create_full_samples(train_data, is_test=False)

# Scaling (use same scaler as before)
scaler = MinMaxScaler().fit(train_samples)

In [ ]:
unit_id = 10
train_data = process_dataset(dataset_test, unit_id=unit_id)

# 3. Full pipeline
# Process raw data without thresholds
train_samples, train_ruls = create_full_samples(train_data, is_test=False)

# Create feature DataFrames
train_feats_df = create_feature_df(model, train_samples, train_ruls, train_loader, scaler)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_matching_columns(df, keywords, r_early=125, figsize=(12, 6), title='Plot of Matching Columns'):
    # Filter the DataFrame to include only the columns that match the keywords
    matching_columns = [col for col in df.columns if any(keyword in col for keyword in keywords)]
    filtered_df = df[matching_columns]

    # Determine the number of entries to plot
    if r_early is not None:
        filtered_df = filtered_df.tail(r_early)

    # Set the style using seaborn
    sns.set(style="whitegrid")

    # Set the font to Times New Roman
    plt.rcParams['font.family'] = 'Times New Roman'
    plt.rcParams['text.color'] = 'black'

    # Plot the matching columns
    filtered_df.plot(figsize=figsize)
    plt.title(title, fontsize=16)
    plt.xlabel('timestep', fontsize=14)
    plt.ylabel('values', fontsize=14)
    plt.legend(loc='upper left', fontsize=14)
    plt.show()

plot_matching_columns(train_feats_df, ['SAP'], figsize=(6, 5), title=f'SAP Encoder HIs for engine 1 on {dataset_name}')
plot_matching_columns(train_feats_df, ['Encoder_HPC', 'Encoder_turbine', 'Encoder_fan'], figsize=(6, 5), title=f'Encoder HIs for engine 1 on {dataset_name}')
plot_matching_columns(train_feats_df, ['fan', 'HPC'], figsize=(6, 5), title=f'Encoder HIs for engine 1 on {dataset_name}')
plot_matching_columns(train_feats_df, ['Encoder_HPC_NAP', 'Encoder_turbine_NAP', 'Encoder_fan_NAP'], figsize=(6, 5), title=f'Encoder NAP HIs for engine 1 on {dataset_name}')
plot_matching_columns(train_feats_df, ['Latent'], figsize=(6, 5), title=f'Latent HI for engine 1 on {dataset_name}')
plot_matching_columns(train_feats_df, ['Aleatoric_HPC', 'Epistemic_HPC', 'Aleatoric_turbine', 'Epistemic_turbine', 'Aleatoric_fan', 'Epistemic_fan'], figsize=(6, 5), title=f'UQ HI for engine 1 on {dataset_name}')
plot_matching_columns(train_feats_df, ['Aleatoric_'], figsize=(6, 5), title=f'Aleatoric UQ HI for engine 1 on {dataset_name}')
plot_matching_columns(train_feats_df, ['Epistemic_HPC', 'Epistemic_fan'], figsize=(6, 5), title=f'Epistemic UQ HI for engine 1 on {dataset_name}')
plot_matching_columns(train_feats_df, ['Epistemic'], figsize=(6, 5), title=f'Epistemic UQ HI for engine 1 on {dataset_name}')